In [1]:
from gurobipy import GRB
import gurobipy as gb

In [2]:
# Should we implement a variable pricing scheme?
isVariablePricing = True

In [3]:
# Linear price response functions (intercept, slope)
response = [[3100, 62], [1900, 50], [1700, 40], [1710, 42], [2000, 53], [2500, 54], [3300, 60]]

In [4]:
# Create a new optimization model to maximize revenue
model = gb.Model("Variable Pricing Model")

Restricted license - for non-production use only - expires 2025-11-24


In [5]:
# Construct the decision variables
p = model.addVars(7, lb=0, ub=40, vtype=GRB.CONTINUOUS, name="Price")
d = model.addVars(7, lb=0, ub=1100, vtype=GRB.CONTINUOUS, name="Daily Demand")

In [6]:
#Objective Function
model.setObjective(gb.quicksum((p[n]-19)*d[n] for n in range(7)), GRB.MAXIMIZE)

In [7]:
# Demand is diverted from days of higher prices to days with lower prices
model.addConstrs((d[n] == response[n][0] - response[n][1]*p[n] + 9*gb.quicksum(p[m] - p[n] for m in range(7)) for n in range(7)), "Demand Constraint")

{0: <gurobi.Constr *Awaiting Model Update*>,
 1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>,
 3: <gurobi.Constr *Awaiting Model Update*>,
 4: <gurobi.Constr *Awaiting Model Update*>,
 5: <gurobi.Constr *Awaiting Model Update*>,
 6: <gurobi.Constr *Awaiting Model Update*>}

In [8]:
# If variable pricing is not allowed, we must add constraints to ensure that
# the price on each day of the week is the same. 
if not isVariablePricing:
    for n in range(6):
        model.addConstr(p[n] == p[n+1], "Equality Constraint %i" %n)

In [9]:
# Solve our model
model.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11.0 (22621.2))

CPU model: 13th Gen Intel(R) Core(TM) i7-13700H, instruction set [SSE2|AVX|AVX2]
Thread count: 14 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 7 rows, 14 columns and 56 nonzeros
Model fingerprint: 0x2ff64bc2
Model has 7 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [2e+01, 2e+01]
  QObjective range [2e+00, 2e+00]
  Bounds range     [4e+01, 1e+03]
  RHS range        [2e+03, 3e+03]

Continuous model is non-convex -- solving as a MIP

Presolve time: 0.00s
Presolved: 22 rows, 22 columns, 99 nonzeros
Presolved model has 7 bilinear constraint(s)
Variable types: 22 continuous, 0 integer (0 binary)
Found heuristic solution: objective 62186.852292

Root relaxation: objective 9.142552e+04, 22 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexp

In [10]:
# Print the objective and decision variables
model.printAttr('X')


    Variable            X 
-------------------------
    Price[0]      33.1479 
    Price[1]      30.3494 
    Price[2]      31.4028 
    Price[3]      31.0332 
    Price[4]      30.2413 
    Price[5]      32.2007 
    Price[6]      34.3454 
Daily Demand[0]          961 
Daily Demand[1]          475 
Daily Demand[2]          470 
Daily Demand[3]          456 
Daily Demand[4]        496.5 
Daily Demand[5]          737 
Daily Demand[6]         1080 
